# Subsequent Memory Effect

### Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


### Settings

In [2]:
# Paths
PROJECT_PATH = 'C:/Users/micha/projects/oscillation_vs_exponent/'


### Load data

In [3]:
# load aggregted spectral parameterization results
data = pd.read_csv(PROJECT_PATH + 'data/results/' + 'df_ols_allchans.csv')
# data = pd.read_csv(PROJECT_PATH + 'data/results/' + 'df_ols.csv')
print(data.shape)
print(len(data)/4)
data.head()

(2780, 20)
695.0


,Unnamed: 0,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,exp_diff,alpha_pre,alpha_post,alpha_diff,peak_present,f_rot,alpha_bp_diff,material,memory,ap_mode
0,0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,-0.037759,0.213201,0.123599,-0.089602,3.0,10.978558,-1536.029206,words,hit,knee
1,1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,3.052480,2.834013,-0.218467,0.210785,0.225154,0.014369,3.0,83.836257,-212.749318,words,hit,knee
2,2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,2.774045,2.725544,-0.048501,0.276397,NaN,NaN,1.0,89.824561,-2992.875343,words,hit,knee
3,3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,2.940543,2.586441,-0.354102,NaN,0.213641,NaN,2.0,59.883041,-1776.054670,words,hit,knee
4,4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,2.872949,2.557193,-0.315757,0.227912,0.182313,-0.045599,3.0,65.871345,-4302.884091,words,hit,knee


### Run 2x2 Anova

In [4]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exp', 'alpha', 'alpha_bp']:
    # Convert NaNs to 0s
    df = data.copy()
    df.loc[np.isnan(df[f'{var}_diff']), f'{var}_diff'] = 0

    # repeated measures anova
    results_i = sm.stats.AnovaRM(df, f"{var}_diff", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.9523 1.0000 694.0000 0.3295
material         4.0674 1.0000 694.0000 0.0441
memory:material  0.0757 1.0000 694.0000 0.7833



===================  alpha  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.0008 1.0000 694.0000 0.9781
material         0.3503 1.0000 694.0000 0.5541
memory:material  0.3869 1.0000 694.0000 0.5342



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.8071 1.0000 694.0000 0.3693
material         4.4911 1.0000 694.0000 0.0344
memory:material  0.0000 1.0000 694.0000 0.9956



Original manuscript applied 2x2 at the level of individual electrodes. This will require further modification of the pipeline

### Restrict analysis to channels exhibiting significant modulation of alpha/beta bandpower

In [5]:
# load task-modulation results
fname_in = r"C:\Users\micha\projects\oscillation_vs_exponent\data\results\ieeg_modulated_channels.csv"
df_tm = pd.read_csv(fname_in)

# modify ccolunm names for consistancy between datframes
df_tm.drop(columns=['Unnamed: 0'], inplace=True)
df_tm.rename(columns={'p_val' : 'pval_tm', 'sign' : 'sign_tm'}, inplace=True)
# for mat in ['words','faces']:
#     df_tm.loc[df_tm['material'] == mat, 'material'] = mat[:-1]

# determine significance
df_tm['sig_tm'] = np.where(df_tm['pval_tm'] < 0.05, 1, 0)

# join
df_merge = pd.merge(data, df_tm, on=['patient','chan_idx','material','memory'])
df_merge

df_merge

,Unnamed: 0,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,...,alpha_diff,peak_present,f_rot,alpha_bp_diff,material,memory,ap_mode,pval_tm,sign_tm,sig_tm
0,0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,...,-0.089602,3.0,10.978558,-1536.029206,words,hit,knee,0.0360,1,1
1,1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,3.052480,2.834013,...,0.014369,3.0,83.836257,-212.749318,words,hit,knee,0.0280,1,1
2,2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,2.774045,2.725544,...,NaN,1.0,89.824561,-2992.875343,words,hit,knee,0.0002,1,1
3,3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,2.940543,2.586441,...,NaN,2.0,59.883041,-1776.054670,words,hit,knee,0.0014,1,1
4,4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,2.872949,2.557193,...,-0.045599,3.0,65.871345,-4302.884091,words,hit,knee,0.0000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,2775,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,2.542326,2.483833,...,NaN,2.0,40.920078,4.359006,faces,miss,knee,0.1656,-1,0
2776,2776,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,3.030214,2.805684,...,NaN,1.0,31.937622,27.284251,faces,miss,knee,0.9758,-1,0
2777,2777,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,3.083662,3.047577,...,NaN,2.0,17.964912,190.037519,faces,miss,knee,0.0140,-1,1
2778,2778,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,2.978981,2.772607,...,NaN,2.0,43.914230,1246.367571,faces,miss,knee,0.0368,-1,1


In [6]:
# find channels that are significant in any condition
df_hit = df_merge[df_merge['memory']=='hit']
sig = df_hit.groupby(['patient','chan_idx']).all().reset_index()

# add to df_merge
df_merge['sig'] = np.nan
for ii in range(len(sig)):
    df_merge.loc[(df_merge['patient']==sig.loc[ii, 'patient']) & \
                (df_merge['chan_idx']==sig.loc[ii, 'chan_idx']), 'sig'] \
                    = sig.loc[ii, 'sig_tm']
    
# reduce
df_sig = df_merge[df_merge['sig']==1]

# show
print(len(df_sig)/4)
df_sig



169.0


,Unnamed: 0,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,...,peak_present,f_rot,alpha_bp_diff,material,memory,ap_mode,pval_tm,sign_tm,sig_tm,sig
0,0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,...,3.0,10.978558,-1536.029206,words,hit,knee,0.0360,1,1,True
1,1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,3.052480,2.834013,...,3.0,83.836257,-212.749318,words,hit,knee,0.0280,1,1,True
2,2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,2.774045,2.725544,...,1.0,89.824561,-2992.875343,words,hit,knee,0.0002,1,1,True
4,4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,2.872949,2.557193,...,3.0,65.871345,-4302.884091,words,hit,knee,0.0000,1,1,True
9,9,pat04,0,A01-A09,-58.30120,52.43075,-26.23685,pat04_0,3.189699,2.446979,...,3.0,36.927875,-49151.042528,words,hit,knee,0.0000,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,2752,pat22,25,PI_05-PI_06,-10.50000,-40.00000,32.50000,pat22_25,4.165989,4.157912,...,0.0,NaN,372.611433,faces,miss,knee,0.0302,-1,1,True
2753,2753,pat22,26,PI_06-PI_07,-11.50000,-40.00000,39.00000,pat22_26,3.771881,3.858311,...,3.0,98.807018,1895.432153,faces,miss,knee,0.1644,-1,0,True
2756,2756,pat22,29,PI_09-PI_10,-11.50000,-41.00000,60.50000,pat22_29,4.296499,4.842026,...,3.0,55.890838,23432.632588,faces,miss,knee,0.1330,-1,0,True
2763,2763,pat22,36,AC_07-AC_08,23.50000,-31.00000,42.00000,pat22_36,3.241910,3.060999,...,3.0,12.974659,686.420920,faces,miss,knee,0.0052,-1,1,True


In [7]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exp', 'alpha', 'alpha_bp']:
    # Convert NaNs to 0s
    df = df_sig.copy()
    df.loc[np.isnan(df[f'{var}_diff']), f'{var}_diff'] = 0

    # repeated measures anova
    results_i = sm.stats.AnovaRM(df, f"{var}_diff", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           4.2915 1.0000 168.0000 0.0398
material         5.7927 1.0000 168.0000 0.0172
memory:material  0.5395 1.0000 168.0000 0.4637



===================  alpha  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.2847 1.0000 168.0000 0.5943
material         0.7782 1.0000 168.0000 0.3789
memory:material  0.0672 1.0000 168.0000 0.7958



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           1.0483 1.0000 168.0000 0.3074
material         0.3091 1.0000 168.0000 0.5790
memory:material  0.7546 1.0000 168.0000 0.3863

